In [1]:
import doctest

In [2]:
def p_equal(p1, p2):
    if len(p1) > 0 and p1[-1] == '$':
        p1 = p1[:-1]
    if len(p2) > 0 and p2[-1] == '$':
        p2 = p2[:-1]
    return p1 == p2

def p_match(p1, p2):
    if p1[-1:] == ['$']:
        if len(p1) != len(p2) or p2[-1:] != ['$']:
            return False
    else:
        if len(p1) > len(p2):
            return False
    return p1 == p2[:len(p1)]

def p_diff(p1, p2):
    for i in range(len(p1)):
        if i >= len(p2) or p1[i] != p2[i]:
            return p1[i:]
    return []

In [3]:
print( p_match(['a','b','c'],['a','b','c','d']) ) #True
print( p_match(['a','$'],['a','b','c']) )         #False
print( p_match(['a','b','c'],['a','c','b']) )     #False
print( p_match(['a','$'],['a']) ) #must return False

True
False
False
False


In [4]:
print( p_diff(['a','b','$'], ['a']) )
print( p_diff(['a','b','d','c'], ['a','b','c']) )
print( p_diff(['a','b','c'], ['a','b','d','c']) )
print( p_diff(['a','c','b','d','$'], ['a','b','c','d']) )
print( p_diff(['a','b','c'], []) )
print( p_diff(['a','b', '$'], ['a', 'b', 'c']) )
print( p_diff(['$'], ['$']) )

['b', '$']
['d', 'c']
['c']
['c', 'b', 'd', '$']
['a', 'b', 'c']
['$']
[]


In [5]:
class Link:
    def __init__(self, test, child):
        self.test = test
        self.child = child
    
    def passes(self, pattern):
        return p_match(self.test, pattern)

In [6]:
class Node:
    idx = 0
    def __init__(self, contents, image, children):
        self.contents = contents
        self.image = image
        self.children = children
        self.idx = Node.idx
        Node.idx += 1

In [17]:
class Chrest:
    def __init__(self):
        self.net = Node(['ROOT NODE'], ['ROOT NODE'], [])

    def recognise(self, pattern):
        if pattern == ['$']:             #Ad-hoc solution to deep children '$' nodes
            return self.net
        current_node = self.net
        children = current_node.children
        sorted_pattern = pattern
        next_link = 0
        
        while next_link < len(children):
            link = children[next_link]
            if link.passes(sorted_pattern):
                current_node = link.child
                children = current_node.children
                next_link = 0
                sorted_pattern = p_diff(sorted_pattern, link.test)
            else:
                next_link += 1
        #print(pattern, link.test, sorted_pattern)
        return current_node
    
    def recognise_and_learn(self, pattern):
        current_node = self.recognise(pattern)
        if current_node.image != pattern:
            if current_node == self.net or \
               not p_match(current_node.image, pattern) or \
               current_node.image[-1:] == ['$']:
                current_node = self.discriminate(current_node, pattern)
            else:
                current_node = self.familiarise(current_node, pattern)
        return current_node
    
    def get_first(self, pattern):
        fst = pattern[:1]
        if len(fst) == 0 or fst == ['$']:
            return []
        else:
            return fst
    
    def familiarise(self, node, pattern):
        new_info = self.get_first(p_diff(pattern, node.image))
        if len(new_info) == 0:
            return node
        
        retrieved_chunk = self.recognise(new_info)
        if retrieved_chunk == self.net:
            return self.learn_primitive(new_info)
        
        if node.image[:-1] == ['$']:
            node.image.pop()
        node.image += new_info
        return node
    
    def discriminate(self, node, pattern):
        new_info = p_diff(pattern, node.contents)
        
        if len(new_info) == 0:
            new_info = ['$']
            if self.recognise(new_info).contents == new_info:
                return self.add_test(node, new_info)
            else:
                child = Node(new_info, new_info, [])
                self.net.children.append(Link(new_info, child))
                return child
        
        chunk = self.recognise(new_info)
        if chunk == self.net:
            return self.learn_primitive(self.get_first(new_info) or ['$'])
        elif p_match(chunk.contents, new_info):
            return self.add_test(node, chunk.contents.copy())
        else:
            return self.add_test(node, self.get_first(new_info))
            

    def add_test(self, node, pattern):
        for child in node.children:
            if child.test == pattern:
                return node
        if node == self.net:
            pat = pattern.copy()
        else:
            pat = node.contents + pattern
        child = Node(pat, pat.copy(), [])
        node.children.append(Link(pattern, child))
        return child
    
    def learn_primitive(self, pattern):
        assert len(pattern) == 1
        child = Node(pattern, [], [])
        self.net.children.append(Link(pattern, child))
        return child
    
    def print_tree(self, node=None, parent=[], level=0):
        if not node:
            node = self.net
        indent = '-------' * level
        contents = '< ' + ' '.join(p_diff(node.contents, parent)) + ' >'
        text = 'Node: ' + str(node.idx)
        image = '< '+ ' '.join(node.image) + ' >'
        print(indent + contents, text, image)
        for child in node.children:
            self.print_tree(child.child, node.contents, level + 1)

In [34]:
def dtest01():
    '''Autimatic Tests
    >>> dtest01()
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B C >
    -------< B > Node: 2 <  >
    -------< C > Node: 3 <  >
    -------< D > Node: 4 < D E >
    --------------< A > Node: 7 < D A >
    -------< E > Node: 5 <  >
    -------< F > Node: 6 <  >
    '''
    
    inp = tuple(map(list, (
        'AB$', 'AB$', 'AB$', 'AB$',
        'ABC$', 'ABC$','ABC$', 'ABC$','ABC$', 'ABC$',
        'DEF$', 'DEF$', 'DEF$', 'DEF$','DEF$',
        'DAC$'
    )))

    Node.idx = 0
    chrest = Chrest()

    for p in inp:
        chrest.recognise_and_learn(p)
        #print(p)
    chrest.print_tree()

#dtest01()
doctest.run_docstring_examples(dtest01, globals(), verbose=True)

Finding tests in NoName
Trying:
    dtest01()
Expecting:
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B C >
    -------< B > Node: 2 <  >
    -------< C > Node: 3 <  >
    -------< D > Node: 4 < D E >
    --------------< A > Node: 7 < D A >
    -------< E > Node: 5 <  >
    -------< F > Node: 6 <  >
ok


In [35]:
def dtest02():
    '''
    >>> dtest02()
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B C >
    --------------< B > Node: 4 < A B C D >
    ---------------------< C > Node: 8 < A B C D E >
    ----------------------------< D > Node: 11 < A B C D >
    -------< B > Node: 2 <  >
    -------< C > Node: 3 <  >
    -------< D > Node: 5 < D Z P >
    -------< Z > Node: 6 <  >
    -------< P > Node: 7 <  >
    -------< $ > Node: 9 <  >
    -------< E > Node: 10 <  >
    '''
    
    inp = tuple(map(list, (
        'AB$', 'AB$', 'AB$', 'AB$', 'AB$', 'AB$',
        'ABC$', 'ABC$', 'ABC$',
        'AB$', 'AB$','AB$', 'AB$',
        'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$', 'DZP$',         #ctrl+/ to comment-uncomment
        'ABCD$', 'ABCD$',
        'ABC$', 'ABC$', 'ABC$',
        'ABCD$', 'ABCD$',
        'ABC$',
        'ABCD$',
        'ABCDE$', 'ABCDE$',
        'ABCD$'
    )))

    Node.idx = 0
    chrest = Chrest()

    for p in inp:
        chrest.recognise_and_learn(p)
        #print(p)
    chrest.print_tree()

#dtest02()
doctest.run_docstring_examples(dtest02, globals(), verbose=True)

Finding tests in NoName
Trying:
    dtest02()
Expecting:
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B C >
    --------------< B > Node: 4 < A B C D >
    ---------------------< C > Node: 8 < A B C D E >
    ----------------------------< D > Node: 11 < A B C D >
    -------< B > Node: 2 <  >
    -------< C > Node: 3 <  >
    -------< D > Node: 5 < D Z P >
    -------< Z > Node: 6 <  >
    -------< P > Node: 7 <  >
    -------< $ > Node: 9 <  >
    -------< E > Node: 10 <  >
ok


In [36]:
def dtest03():
    '''
    >>> dtest03()
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B C >
    --------------< B > Node: 4 < A B >
    -------< B > Node: 2 <  >
    -------< C > Node: 3 <  >
    -------< D > Node: 5 < D E >
    --------------< A B > Node: 7 < D A B >
    -------< E > Node: 6 <  >

    '''
    inp = tuple(map(list, (
        'ABC$', 'ABC$', 'ABC$', 'ABC$', 'ABC$', 'ABC$',
        'AB$',
        'DEF$', 'DEF$', 'DEF$', 'DEF$',
        'DAB$'
    )))

    Node.idx = 0
    chrest = Chrest()

    for p in inp:
        chrest.recognise_and_learn(p)
        #print(p)
    chrest.print_tree()

#dtest03()
doctest.run_docstring_examples(dtest03, globals(), verbose=True)

Finding tests in NoName
Trying:
    dtest03()
Expecting:
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B C >
    --------------< B > Node: 4 < A B >
    -------< B > Node: 2 <  >
    -------< C > Node: 3 <  >
    -------< D > Node: 5 < D E >
    --------------< A B > Node: 7 < D A B >
    -------< E > Node: 6 <  >
ok


In [37]:
def dtest04():
    '''
    >>> dtest04()
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B >
    --------------< C > Node: 5 < A C >
    -------< B > Node: 2 <  >
    -------< $ > Node: 3 <  >
    -------< C > Node: 4 <  >
    -------< $ > Node: 6 <  >

    '''
    inp = tuple(map(list, (
        'AB$', 'AB$', 'AB$', 'AB$',
        'A$',
        'AC$','AC$',
        'A$'

    )))

    Node.idx = 0
    chrest = Chrest()

    for p in inp:
        chrest.recognise_and_learn(p)
        #print(p)
    chrest.print_tree()

#dtest04()
doctest.run_docstring_examples(dtest04, globals(), verbose=True)

Finding tests in NoName
Trying:
    dtest04()
Expecting:
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B >
    --------------< C > Node: 5 < A C >
    -------< B > Node: 2 <  >
    -------< $ > Node: 3 <  >
    -------< C > Node: 4 <  >
    -------< $ > Node: 6 <  >
ok


In [38]:
def dtest05():
    '''
    >>> dtest05()
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B >
    --------------< C > Node: 4 < A C >
    -------< B > Node: 2 < B B >
    --------------< A C > Node: 5 < B A C >
    -------< C > Node: 3 <  >

    '''
    
    inp = tuple(map(list, (
        'AB$', 'AB$', 'AB$', 'AB$','AB$',
        'AC$', 'AC$', 'AC$', 'AC$', 
        'BB$', 'BB$',
        'BAC$'
    )))

    Node.idx = 0
    chrest = Chrest()

    for p in inp:
        chrest.recognise_and_learn(p)
        #print(p)
    chrest.print_tree()

#dtest05()
doctest.run_docstring_examples(dtest05, globals(), verbose=True)

Finding tests in NoName
Trying:
    dtest05()
Expecting:
    < ROOT NODE > Node: 0 < ROOT NODE >
    -------< A > Node: 1 < A B >
    --------------< C > Node: 4 < A C >
    -------< B > Node: 2 < B B >
    --------------< A C > Node: 5 < B A C >
    -------< C > Node: 3 <  >
ok
